In [1]:
import os
from time import time
import yaml

from argparse import ArgumentParser, Namespace

from s3ts.api.nets.wrapper import WrapperModel
from s3ts.helper_functions import load_dm, str_time, get_parser

from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

from s3ts.helper_functions import get_model

import torch
import numpy as np

import matplotlib.pyplot as plt

In [2]:
p = get_parser()

In [3]:
args = p.parse_args("--mode ts --dataset UCI-HAR --lr 0.001 --subjects_for_test 21 --window_size 25 --window_stride 1 --batch_size 128 --encoder_architecture cnn_gap --encoder_features 20 --decoder_architecture mlp --decoder_features 32 --decoder_layers 1 --pattern_size 32 --rho 0.1 --compute_n 300 --pattern_type syn --num_workers 8 --max_epochs 3 --normalize --reduce_imbalance --training_dir training_cv2 --n_val_subjects 4 --weight_decayL1 0 --weight_decayL2 0".split())

In [4]:
dm = load_dm(args)

class_changes = [0] + list(np.nonzero(np.diff(dm.stsds.SCS))[0])
print(len(class_changes), "number of class changes")

Loaded dataset UCI-HAR with a total of 815614 observations for window size 25
Sampling 59084 (balanced) observations per epoch.
Using 708997 observations for training, 106617 for validation and 25512 observations for test
1787 number of class changes


In [8]:
fft_res = np.fft.fft(dm.stsds.STS[0, (class_changes[2]+1):(class_changes[3]+1)], n=512)

In [52]:
def process_fft(STS, SCS):
    class_changes = [0] + list(np.nonzero(np.diff(SCS))[0])

    # top10 = {} # a dict for each class
    magnitudes = {} # a dict for each class
    classes = np.unique(SCS)
    for c in classes:
        # top10[c] = [{} for i in range(STS.shape[0])] # a dict for each channel
        magnitudes[c] = [{} for i in range(STS.shape[0])] # a dict for each channel

    for i in range(len(class_changes)-1):
        current_class = SCS[class_changes[i]+1].item()

        series_part = STS[:, (class_changes[i]+1):(class_changes[i+1]+1)]
        fft_size = 2**int(np.log2(series_part.shape[1]))
        fft_short = np.fft.fft(series_part, axis=-1, n=fft_size)
        fft_freq = np.fft.fftfreq(fft_size) # highest frequencies for signals of sampling rate 50 is 25

        # fft_short_sort = np.argsort(np.abs(fft_short), axis=-1) # sort is ascending magnitudes

        # fft_freq_sort = np.abs(fft_freq[fft_short_sort][:,-5:]) # get the 10 for each channel frequencies (+-) with highest amplitude

        for c in range(fft_short.shape[0]):
            for j in range(fft_short.shape[1]):
                magnitudes[current_class][c][fft_freq[j]] = magnitudes[current_class][c].get(fft_freq[j], 0) + np.abs(fft_short[c, j])

            # for j in range(fft_freq_sort.shape[1]):
            #     top10[current_class][c][fft_freq_sort[c, j]] = top10[current_class][c].get(fft_freq_sort[c, j], 0) + 1

    return magnitudes

In [49]:
def get_predominant_frequency(fft_mag, mode="count"):
    classes_list = list(filter(lambda x: x!=100, fft_mag.keys()))
    num_classes = len(classes_list)

    if mode=="count":
        out = np.zeros((num_classes, len(fft_mag[0]))) # (n, c) we get a predominant frequency per channel, per class

        for i, c in enumerate(classes_list):
            for j, channel_result in enumerate(fft_mag[c]):
                sorted_fr = list(filter(lambda x: x[0]>0, sorted(channel_result.items(), key=lambda x:x[1])))
                out[i, j] = sorted_fr[-1][0]
        
        return out
    
    elif mode=="magnitude": # get the frequencies with most importance in the fft transform
        out = {} # frequencies magnitude total

        for i, c in enumerate(classes_list):
            for j, channel_result in enumerate(fft_mag[c]):
                for f in channel_result.keys():
                     out[f] = out.get(f, 0) + channel_result[f]                
        
        frequencies_ordered = np.array(list(filter(lambda x: x[0]>0, sorted(out.items(), key=lambda x:x[1], reverse=True))))

        return frequencies_ordered

In [53]:
res = process_fft(dm.stsds.STS, dm.stsds.SCS)

In [50]:
pred_f = get_predominant_frequency(res[0], res[1], "magnitude")

In [51]:
pred_f

array([[1.56250000e-02, 1.27775835e+04],
       [7.81250000e-03, 1.14858395e+04],
       [3.12500000e-02, 1.02229223e+04],
       ...,
       [3.72070312e-01, 2.52868040e+01],
       [4.95117188e-01, 2.24361509e+01],
       [4.26757812e-01, 2.01007423e+01]])